In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt

from skimage.feature import hog
from skimage.color import rgb2grey
import cv2 as cv

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc

In [2]:
# by github/jrjohansson
# ! pip install --user version_information
%load_ext version_information
%version_information pandas, matplotlib, sklearn, scipy, tensorflow, cv2

The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information


Software,Version
Python,3.7.4 64bit [Clang 4.0.1 (tags/RELEASE_401/final)]
IPython,7.12.0
OS,Darwin 19.3.0 x86_64 i386 64bit
jupyter,The 'jupyter' distribution was not found and is required by the application
pandas,1.0.0
matplotlib,3.1.3
sklearn,0.21.2
scipy,1.4.1
tensorflow,1.15.0
cv2,4.1.0


In [2]:
matTr = loadmat('train_32x32.mat')
matTe = loadmat('test_32x32.mat')
# matExt = loadmat('extra_32x32.mat')

# labels are originally in [1,10] and now will be in [0,9]
Xtr, Ytr = matTr['X'], matTr['y']-1
Xte, Yte = matTe['X'], matTe['y']-1
# Xext, Yext = matExt['X'], matExt['y']-1

# changing the dimensions so that the number of the input image is the first
Xtr = np.transpose(Xtr, (3, 0, 1, 2))
Xte = np.transpose(Xte, (3, 0, 1, 2))
# Xext = np.transpose(Xext, (3, 0, 1, 2))

# Xtr_ext = np.concatenate((Xtr,Xext))
# Ytr_ext = np.concatenate((Ytr,Yext))

# Xtr_ext = Xtr_ext / 255.0
Ytr = np.squeeze(Ytr)
Yte = np.squeeze(Yte)
# Yext = np.squeeze(Yext)

In [3]:
batch_size = 128
epochs = 15
IMG_HEIGHT = 32
IMG_WIDTH = 32
NUM_CHANNEL = 3

In [4]:
# Xtr_gray = np.zeros((len(Xtr),32,32,1))
# for i in range(len(Xtr)):
#     Xtr_gray[i] = tf.image.rgb_to_grayscale(Xtr[i])
Xtr_gray = tf.image.rgb_to_grayscale(Xtr)


In [5]:
Xtr_gray = K.eval(Xtr_gray)

In [6]:
Xtr_bin = np.zeros((len(Xtr_gray),32,32,1))
for i in range(len(Xtr_gray)):
    Xtr_bin[i,:,:,0] = cv.adaptiveThreshold(Xtr_gray[i,:,:,0],255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,cv.THRESH_BINARY,11,2)
#     blur = cv.GaussianBlur(Xtr_gray[i,:,:,0],(5,5),0)
#     _, Xtr_bin[i,:,:,0] = cv.threshold(blur,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)

```python
ind = 700
titles = ['RGB Image', 'GRAY_SCALE', 'BINARY']
images = [Xtr[ind], Xtr_gray[ind,:,:,0], Xtr_bin[ind,:,:,0]]
for i in range(3):
    plt.subplot(1,3,i+1)
    if i!=0:
        plt.imshow(images[i],'gray')
    else:
        plt.imshow(images[i])
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])
plt.savefig('images.png',bbox_inches = 'tight')
plt.show()
```

```python
plt.hist(Ytr, ec='k',bins=10)
plt.title('Histogram of Train Data')
plt.savefig('histogram.png',bbox_inches = 'tight')
```

In [ ]:
# model = Sequential()
# model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(32,32,3)))
# model.add(BatchNormalization())
# model.add(Conv2D(32, 3, activation='relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.4))

# model.add(Conv2D(64, kernel_size = 3, activation='relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(64, kernel_size = 3, activation='relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.4))

# model.add(Conv2D(128, kernel_size = 4, activation='relu'))
# model.add(BatchNormalization())
# model.add(Flatten())
# model.add(Dropout(0.4))
# model.add(Dense(10, activation='softmax'))

# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

In [12]:
nets = 5
model = [0]*nets
history = [0]*nets

In [23]:
for i in range(3):
    model[i] = Sequential()
    model[i].add(Conv2D(24, kernel_size=5, activation='relu', padding="same", input_shape=(32,32,1)))
    model[i].add(MaxPooling2D(pool_size=(2,2)))
    if i>0:
        model[i].add(Conv2D(48, kernel_size=5, activation='relu', padding="same"))
        model[i].add(MaxPooling2D(pool_size=(2,2)))
    if i>1:
        model[i].add(Conv2D(64, kernel_initializer='he_normal', kernel_size=5, activation='relu', padding="same"))
        model[i].add(MaxPooling2D(pool_size=(2,2), padding='same'))
    model[i].add(Flatten())
    model[i].add(Dense(256, activation='relu'))
    model[i].add(Dense(10, activation='softmax'))
    myAdam = tf.keras.optimizers.Adam(learning_rate=0.00001)
    model[i].compile(optimizer=myAdam,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

In [ ]:
# # CREATE MORE IMAGES VIA DATA AUGMENTATION
# datagen = ImageDataGenerator(
#         rotation_range=10,  
#         zoom_range = 0.10,  
#         width_shift_range=0.1, 
#         height_shift_range=0.1)

In [ ]:
model.summary()

In [24]:
# for j in range(nets):
#     X_train2, X_val2, Y_train2, Y_val2 = train_test_split(Xtr, Ytr, test_size = 0.1)
#     history[j] = model[j].fit_generator(datagen.flow(X_train2,Y_train2, batch_size=64),
#         epochs = epochs, steps_per_epoch = X_train2.shape[0]//64,  
#         validation_data = (X_val2,Y_val2), callbacks=[annealer], verbose=0)
#     print("CNN {0:d}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
#         j+1,epochs,max(history[j].history['acc']),max(history[j].history['val_acc']) ))
j=0
epochs = 20
# DECREASE LEARNING RATE EACH EPOCH
# annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(Xtr_bin, Ytr, test_size = 0.1)
history[j] = model[j].fit(X_train2, Y_train2, epochs=20, batch_size=64,
                          validation_data = (X_val2, Y_val2), verbose=2)
print("CNN {0:d}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
        j+1,epochs,max(history[j].history['acc']),max(history[j].history['val_acc']) ))

Train on 65931 samples, validate on 7326 samples
Epoch 1/20
65931/65931 - 35s - loss: 2.3696 - acc: 0.0914 - val_loss: 2.3700 - val_acc: 0.0912
Epoch 2/20
65931/65931 - 37s - loss: 2.3669 - acc: 0.0943 - val_loss: 2.3700 - val_acc: 0.0912
Epoch 3/20
65931/65931 - 35s - loss: 2.3669 - acc: 0.0943 - val_loss: 2.3700 - val_acc: 0.0912
Epoch 4/20


KeyboardInterrupt: 

In [ ]:
# # DECREASE LEARNING RATE EACH EPOCH
# annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)


# epochs=50
# X_train2, X_val2, Y_train2, Y_val2 = train_test_split(Xtr, Ytr, test_size = 0.1)
# history = model.fit_generator(datagen.flow(X_train2,Y_train2, batch_size=64),
#                               epochs = epochs, steps_per_epoch = X_train2.shape[0]//64,  
#                               validation_data = (X_val2,Y_val2), callbacks=[annealer], verbose=0)
# print("CNN: Epochs={0:d}, Train accuracy={1:.5f}, Validation accuracy={2:.5f}".format(
#     epochs,history['acc'],history['val_acc']))

# ML model

In [ ]:
def create_features(img):
    # flatten three channel color image
    color_features = img.flatten()
    # convert image to greyscale
    grey_image = rgb2grey(img)
    # get HOG features from greyscale image
    hog_features, _ = hog(img, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)
    # combine color and hog features into a single array
    flat_features = np.hstack((color_features,hog_features))
    return hog_features

In [ ]:
features_list = []
for i in range(len(Xtr)):
    image_features = create_features(Xtr[i])
    features_list.append(image_features)  
# convert list of arrays into a matrix
feature_matrix = np.array(features_list)


In [ ]:
feature_matrix.shape

In [ ]:
# scaling the features
from sklearn.preprocessing import scale
X_scaled = scale(feature_matrix)

# train test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Ytr, test_size = 0.3, train_size = 0.2 ,random_state = 10)

In [ ]:

model_linear = SVC(kernel='linear')
model_linear.fit(X_train, y_train)

# predict
y_pred = model_linear.predict(X_test)

In [ ]:
# confusion matrix and accuracy

from sklearn import metrics
from sklearn.metrics import confusion_matrix
# accuracy
print("accuracy:", metrics.accuracy_score(y_true=y_test, y_pred=y_pred), "\n")

# cm
print(metrics.confusion_matrix(y_true=y_test, y_pred=y_pred))